# Google Flights Airfare Forecast - Data Analysis & Model Development

This comprehensive notebook explores the Google Flights airfare dataset, performs feature engineering, trains multiple machine learning models, and provides business insights for flight price prediction.

## Objectives:
1. **Explore and understand** the flight pricing data
2. **Engineer meaningful features** for price prediction
3. **Train and compare** multiple ML models
4. **Analyze model performance** and feature importance
5. **Generate business insights** for optimal booking strategies

---

## 1. Import Required Libraries

Import all necessary libraries for data analysis, visualization, and machine learning.

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.inspection import permutation_importance
import xgboost as xgb

# Statistical analysis
from scipy import stats
from scipy.stats import pearsonr

# System and file operations
import sys
import os
from pathlib import Path

# Add project root to path
project_root = Path().absolute().parent
sys.path.append(str(project_root))

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📚 All libraries imported successfully!")
print(f"🐍 Python version: {sys.version}")
print(f"🐼 Pandas version: {pd.__version__}")
print(f"📊 NumPy version: {np.__version__}")

## 2. Load and Inspect Data

Load the Google Flights airfare dataset and perform initial data exploration.

In [ ]:
# Load the dataset
data_path = '../data/google_flights_airfare_data.csv'
df = pd.read_csv(data_path)

print(f"📊 Dataset loaded successfully!")
print(f"📏 Dataset shape: {df.shape}")
print(f"💾 Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Display basic information
print("\n" + "="*50)
print("BASIC DATASET INFORMATION")
print("="*50)
df.info()

In [ ]:
# Display first few rows
print("📋 First 5 rows of the dataset:")
display(df.head())

print("\n📋 Last 5 rows of the dataset:")
display(df.tail())

# Basic statistics
print("\n📊 Statistical Summary:")
display(df.describe())

# Check for missing values
print("\n🔍 Missing Values:")
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({
    'Column': missing_values.index,
    'Missing Count': missing_values.values,
    'Missing Percentage': missing_percent.values
})
display(missing_df[missing_df['Missing Count'] > 0])

In [ ]:
# Unique values analysis
print("🔢 Unique Values Analysis:")
print(f"Airlines: {df['airline'].nunique()}")
print(f"Origins: {df['origin'].nunique()}")
print(f"Destinations: {df['destination'].nunique()}")
print(f"Fare Classes: {df['fare_class'].nunique()}")
print(f"Flight IDs: {df['flight_id'].nunique()}")

print(f"\n✈️ Airlines in dataset:")
print(df['airline'].value_counts())

print(f"\n🎫 Fare Classes:")
print(df['fare_class'].value_counts())

print(f"\n🏢 Top 10 Origins:")
print(df['origin'].value_counts().head(10))

print(f"\n🏢 Top 10 Destinations:")
print(df['destination'].value_counts().head(10))

## 3. Data Preprocessing

Clean and prepare the data for analysis and modeling.

In [ ]:
# Create a copy for preprocessing
df_processed = df.copy()

# Convert datetime columns
print("📅 Converting datetime columns...")
df_processed['booking_dt'] = pd.to_datetime(df_processed['booking_dt'])
df_processed['departure_dt'] = pd.to_datetime(df_processed['departure_dt'])

# Check for invalid dates
invalid_dates = df_processed[df_processed['departure_dt'] <= df_processed['booking_dt']]
print(f"⚠️ Found {len(invalid_dates)} records with departure date <= booking date")

# Remove invalid records
df_processed = df_processed[df_processed['departure_dt'] > df_processed['booking_dt']]
print(f"✅ After filtering: {len(df_processed)} records remain")

# Check for outliers in price
Q1 = df_processed['price'].quantile(0.25)
Q3 = df_processed['price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df_processed[(df_processed['price'] < lower_bound) | (df_processed['price'] > upper_bound)]
print(f"📊 Price outliers detected: {len(outliers)} ({len(outliers)/len(df_processed)*100:.2f}%)")
print(f"📊 Price range: ${df_processed['price'].min():.2f} - ${df_processed['price'].max():.2f}")
print(f"📊 IQR bounds: ${lower_bound:.2f} - ${upper_bound:.2f}")

# Check data quality
print(f"\n🔍 Data Quality Check:")
print(f"Duplicate rows: {df_processed.duplicated().sum()}")
print(f"Records with price <= 0: {(df_processed['price'] <= 0).sum()}")
print(f"Records with negative advance days: {((df_processed['departure_dt'] - df_processed['booking_dt']).dt.days < 0).sum()}")

## 4. Feature Engineering

Create meaningful features from the raw data to improve model performance.

In [ ]:
# Feature Engineering
print("🔧 Creating engineered features...")

# 1. Temporal Features
df_processed['days_before_departure'] = (df_processed['departure_dt'] - df_processed['booking_dt']).dt.days

# Booking date features
df_processed['booking_month'] = df_processed['booking_dt'].dt.month
df_processed['booking_day_of_week'] = df_processed['booking_dt'].dt.dayofweek
df_processed['booking_day_of_year'] = df_processed['booking_dt'].dt.dayofyear
df_processed['booking_hour'] = df_processed['booking_dt'].dt.hour
df_processed['booking_quarter'] = df_processed['booking_dt'].dt.quarter

# Departure date features
df_processed['departure_month'] = df_processed['departure_dt'].dt.month
df_processed['departure_day_of_week'] = df_processed['departure_dt'].dt.dayofweek
df_processed['departure_hour'] = df_processed['departure_dt'].dt.hour
df_processed['departure_quarter'] = df_processed['departure_dt'].dt.quarter

# 2. Seasonal Features
df_processed['is_weekend_departure'] = df_processed['departure_day_of_week'].isin([5, 6]).astype(int)
df_processed['is_weekend_booking'] = df_processed['booking_day_of_week'].isin([5, 6]).astype(int)
df_processed['is_summer'] = df_processed['departure_month'].isin([6, 7, 8]).astype(int)
df_processed['is_winter_holidays'] = df_processed['departure_month'].isin([12, 1]).astype(int)
df_processed['is_spring_break'] = df_processed['departure_month'].isin([3, 4]).astype(int)

# 3. Route Features
df_processed['route'] = df_processed['origin'] + '_' + df_processed['destination']
major_hubs = ['ORD', 'ATL', 'LAX', 'DFW', 'DEN', 'JFK', 'SFO', 'LAS', 'PHX', 'CLT']
df_processed['origin_is_hub'] = df_processed['origin'].isin(major_hubs).astype(int)
df_processed['destination_is_hub'] = df_processed['destination'].isin(major_hubs).astype(int)

# Route popularity
route_popularity = df_processed['route'].value_counts() / len(df_processed)
df_processed['route_popularity'] = df_processed['route'].map(route_popularity)

# 4. Airline Features
legacy_carriers = ['American Airlines', 'Delta Air Lines', 'United Airlines']
df_processed['is_legacy_carrier'] = df_processed['airline'].isin(legacy_carriers).astype(int)

# Airline market share
airline_market_share = df_processed['airline'].value_counts() / len(df_processed)
df_processed['airline_market_share'] = df_processed['airline'].map(airline_market_share)

# 5. Advance Booking Categories
df_processed['booking_category'] = pd.cut(
    df_processed['days_before_departure'], 
    bins=[-1, 0, 7, 14, 30, 60, 90, float('inf')],
    labels=['same_day', 'week', '2weeks', 'month', '2months', '3months', 'advance']
)

print(f"✅ Feature engineering completed!")
print(f"📊 Total features created: {len(df_processed.columns) - len(df.columns)}")
print(f"📊 Current dataset shape: {df_processed.shape}")

# Display new features summary
new_features = [col for col in df_processed.columns if col not in df.columns]
print(f"\n🆕 New features created: {new_features}")

## 5. Exploratory Data Analysis & Visualization

Visualize patterns and relationships in the data to gain insights.

In [ ]:
# Price Distribution Analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Price histogram
axes[0, 0].hist(df_processed['price'], bins=50, alpha=0.7, edgecolor='black')
axes[0, 0].set_title('Price Distribution', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Price ($)')
axes[0, 0].set_ylabel('Frequency')

# Log price distribution
axes[0, 1].hist(np.log1p(df_processed['price']), bins=50, alpha=0.7, edgecolor='black', color='orange')
axes[0, 1].set_title('Log Price Distribution', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Log(Price + 1)')
axes[0, 1].set_ylabel('Frequency')

# Price by fare class
df_processed.boxplot(column='price', by='fare_class', ax=axes[1, 0])
axes[1, 0].set_title('Price Distribution by Fare Class', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Fare Class')
axes[1, 0].set_ylabel('Price ($)')

# Price by airline
top_airlines = df_processed['airline'].value_counts().head(5).index
df_top_airlines = df_processed[df_processed['airline'].isin(top_airlines)]
df_top_airlines.boxplot(column='price', by='airline', ax=axes[1, 1])
axes[1, 1].set_title('Price Distribution by Top Airlines', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Airline')
axes[1, 1].set_ylabel('Price ($)')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Summary statistics by key categories
print("📊 Price Statistics by Fare Class:")
print(df_processed.groupby('fare_class')['price'].agg(['count', 'mean', 'median', 'std']).round(2))

print("\n📊 Price Statistics by Top Airlines:")
print(df_processed[df_processed['airline'].isin(top_airlines)].groupby('airline')['price'].agg(['count', 'mean', 'median', 'std']).round(2))

In [ ]:
# Advance Booking Analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Days before departure vs Price
sample_data = df_processed.sample(min(10000, len(df_processed)))  # Sample for better visualization
axes[0, 0].scatter(sample_data['days_before_departure'], sample_data['price'], alpha=0.5, s=1)
axes[0, 0].set_title('Price vs Days Before Departure', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Days Before Departure')
axes[0, 0].set_ylabel('Price ($)')

# Average price by advance booking category
booking_cat_price = df_processed.groupby('booking_category')['price'].mean().reset_index()
axes[0, 1].bar(booking_cat_price['booking_category'], booking_cat_price['price'])
axes[0, 1].set_title('Average Price by Booking Category', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Booking Category')
axes[0, 1].set_ylabel('Average Price ($)')
axes[0, 1].tick_params(axis='x', rotation=45)

# Price by day of week (departure)
dow_price = df_processed.groupby('departure_day_of_week')['price'].mean().reset_index()
dow_labels = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
axes[1, 0].bar(range(7), dow_price['price'])
axes[1, 0].set_title('Average Price by Departure Day of Week', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Day of Week')
axes[1, 0].set_ylabel('Average Price ($)')
axes[1, 0].set_xticks(range(7))
axes[1, 0].set_xticklabels(dow_labels)

# Price by month (departure)
month_price = df_processed.groupby('departure_month')['price'].mean().reset_index()
axes[1, 1].plot(month_price['departure_month'], month_price['price'], marker='o', linewidth=2, markersize=8)
axes[1, 1].set_title('Average Price by Departure Month', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Month')
axes[1, 1].set_ylabel('Average Price ($)')
axes[1, 1].set_xticks(range(1, 13))
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Correlation analysis
correlation_features = ['price', 'days_before_departure', 'booking_month', 'departure_month', 
                       'booking_day_of_week', 'departure_day_of_week', 'is_weekend_departure',
                       'is_legacy_carrier', 'origin_is_hub', 'destination_is_hub', 'route_popularity']

correlation_matrix = df_processed[correlation_features].corr()

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f', cbar_kws={'shrink': 0.8})
plt.title('Feature Correlation Matrix', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("🔍 Key Correlations with Price:")
price_correlations = correlation_matrix['price'].abs().sort_values(ascending=False)
for feature, corr in price_correlations.items():
    if feature != 'price':
        print(f"   {feature}: {corr:.3f}")

## 6. Model Training

Prepare data for modeling and train multiple machine learning algorithms.

In [ ]:
# Prepare data for modeling
print("🛠️ Preparing data for modeling...")

# Select features for modeling
feature_columns = [
    'days_before_departure', 'booking_month', 'booking_day_of_week', 'booking_hour', 'booking_quarter',
    'departure_month', 'departure_day_of_week', 'departure_hour', 'departure_quarter',
    'is_weekend_departure', 'is_weekend_booking', 'is_summer', 'is_winter_holidays', 'is_spring_break',
    'origin_is_hub', 'destination_is_hub', 'route_popularity', 'is_legacy_carrier', 'airline_market_share'
]

# Encode categorical variables
categorical_columns = ['airline', 'origin', 'destination', 'fare_class', 'booking_category']
le_dict = {}
df_model = df_processed.copy()

for col in categorical_columns:
    if col in df_model.columns:
        le = LabelEncoder()
        df_model[f'{col}_encoded'] = le.fit_transform(df_model[col])
        le_dict[col] = le
        feature_columns.append(f'{col}_encoded')

# Prepare features and target
X = df_model[feature_columns].fillna(0)
y = df_model['price']

print(f"📊 Features: {len(feature_columns)}")
print(f"📊 Samples: {len(X)}")
print(f"📊 Feature columns: {feature_columns}")

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"✅ Data split completed:")
print(f"   Training set: {X_train.shape}")
print(f"   Test set: {X_test.shape}")

# Scale features for linear models
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Train multiple models
print("🤖 Training multiple models...")

models = {
    'XGBoost': xgb.XGBRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, random_state=42),
    'Ridge Regression': Ridge(alpha=1.0)
}

model_results = {}

for name, model in models.items():
    print(f"\n🔄 Training {name}...")
    
    # Use scaled data for Ridge Regression
    if name == 'Ridge Regression':
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        
        # Cross-validation
        cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='neg_mean_absolute_error')
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        # Cross-validation
        cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
    
    # Calculate metrics
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    
    model_results[name] = {
        'model': model,
        'mae': mae,
        'mse': mse,
        'rmse': rmse,
        'r2': r2,
        'mape': mape,
        'cv_mae': -cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'predictions': y_pred
    }
    
    print(f"   ✅ {name} completed:")
    print(f"      MAE: ${mae:.2f}")
    print(f"      RMSE: ${rmse:.2f}")
    print(f"      R²: {r2:.4f}")
    print(f"      MAPE: {mape:.2f}%")
    print(f"      CV MAE: ${-cv_scores.mean():.2f} (±{cv_scores.std():.2f})")

print("\n🏆 Model Performance Summary:")
print("="*80)
results_df = pd.DataFrame({
    name: {
        'MAE': f"${results['mae']:.2f}",
        'RMSE': f"${results['rmse']:.2f}",
        'R²': f"{results['r2']:.4f}",
        'MAPE': f"{results['mape']:.2f}%",
        'CV MAE': f"${results['cv_mae']:.2f}"
    }
    for name, results in model_results.items()
}).T

display(results_df)

## 7. Model Evaluation

Comprehensive evaluation of trained models with visualizations and feature importance analysis.

In [ ]:
# Model Evaluation Visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Choose best model for detailed analysis
best_model_name = min(model_results.keys(), key=lambda x: model_results[x]['mae'])
best_model = model_results[best_model_name]
print(f"🏆 Best performing model: {best_model_name}")

# 1. Actual vs Predicted scatter plot
axes[0, 0].scatter(y_test, best_model['predictions'], alpha=0.5, s=1)
axes[0, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0, 0].set_xlabel('Actual Price ($)')
axes[0, 0].set_ylabel('Predicted Price ($)')
axes[0, 0].set_title(f'{best_model_name}: Actual vs Predicted', fontsize=14, fontweight='bold')

# 2. Residuals plot
residuals = y_test - best_model['predictions']
axes[0, 1].scatter(best_model['predictions'], residuals, alpha=0.5, s=1)
axes[0, 1].axhline(y=0, color='r', linestyle='--')
axes[0, 1].set_xlabel('Predicted Price ($)')
axes[0, 1].set_ylabel('Residuals ($)')
axes[0, 1].set_title(f'{best_model_name}: Residuals Plot', fontsize=14, fontweight='bold')

# 3. Model comparison (MAE)
model_names = list(model_results.keys())
mae_values = [model_results[name]['mae'] for name in model_names]
axes[1, 0].bar(model_names, mae_values)
axes[1, 0].set_title('Model Comparison: Mean Absolute Error', fontsize=14, fontweight='bold')
axes[1, 0].set_ylabel('MAE ($)')
axes[1, 0].tick_params(axis='x', rotation=45)

# 4. Model comparison (R²)
r2_values = [model_results[name]['r2'] for name in model_names]
axes[1, 1].bar(model_names, r2_values)
axes[1, 1].set_title('Model Comparison: R² Score', fontsize=14, fontweight='bold')
axes[1, 1].set_ylabel('R² Score')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Prediction error analysis
print(f"\n📊 Prediction Error Analysis for {best_model_name}:")
print(f"   Mean residual: ${residuals.mean():.2f}")
print(f"   Std residual: ${residuals.std():.2f}")
print(f"   Within $50: {(np.abs(residuals) <= 50).mean()*100:.1f}%")
print(f"   Within $100: {(np.abs(residuals) <= 100).mean()*100:.1f}%")
print(f"   Within $200: {(np.abs(residuals) <= 200).mean()*100:.1f}%")

In [ ]:
# Feature Importance Analysis
if hasattr(best_model['model'], 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': feature_columns,
        'importance': best_model['model'].feature_importances_
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(12, 8))
    sns.barplot(data=feature_importance.head(15), x='importance', y='feature')
    plt.title(f'{best_model_name}: Top 15 Feature Importances', fontsize=16, fontweight='bold')
    plt.xlabel('Importance')
    plt.tight_layout()
    plt.show()
    
    print(f"🔍 Top 10 Most Important Features ({best_model_name}):")
    for i, (_, row) in enumerate(feature_importance.head(10).iterrows(), 1):
        print(f"   {i:2d}. {row['feature']}: {row['importance']:.4f}")
else:
    print(f"⚠️ {best_model_name} does not provide feature importance")

# Permutation importance for all models
print(f"\n🔍 Permutation Importance Analysis:")
for name, results in model_results.items():
    if name == best_model_name:  # Only for best model to save time
        print(f"\n   Calculating permutation importance for {name}...")
        
        if name == 'Ridge Regression':
            perm_importance = permutation_importance(results['model'], X_test_scaled, y_test, 
                                                   n_repeats=5, random_state=42, scoring='neg_mean_absolute_error')
        else:
            perm_importance = permutation_importance(results['model'], X_test, y_test, 
                                                   n_repeats=5, random_state=42, scoring='neg_mean_absolute_error')
        
        perm_df = pd.DataFrame({
            'feature': feature_columns,
            'importance_mean': perm_importance.importances_mean,
            'importance_std': perm_importance.importances_std
        }).sort_values('importance_mean', ascending=False)
        
        print(f"   Top 5 features by permutation importance:")
        for i, (_, row) in enumerate(perm_df.head(5).iterrows(), 1):
            print(f"      {i}. {row['feature']}: {row['importance_mean']:.4f} (±{row['importance_std']:.4f})")

## 8. Make Predictions

Demonstrate how to use the trained model for new predictions and scenario planning.

In [ ]:
# Example Predictions
print("🔮 Making example predictions...")

def create_prediction_features(airline, origin, destination, booking_date, departure_date, fare_class):
    """Create feature vector for prediction"""
    
    # Calculate temporal features
    days_before = (departure_date - booking_date).days
    
    features = {
        'days_before_departure': days_before,
        'booking_month': booking_date.month,
        'booking_day_of_week': booking_date.weekday(),
        'booking_hour': booking_date.hour,
        'booking_quarter': (booking_date.month - 1) // 3 + 1,
        'departure_month': departure_date.month,
        'departure_day_of_week': departure_date.weekday(),
        'departure_hour': departure_date.hour,
        'departure_quarter': (departure_date.month - 1) // 3 + 1,
        'is_weekend_departure': 1 if departure_date.weekday() >= 5 else 0,
        'is_weekend_booking': 1 if booking_date.weekday() >= 5 else 0,
        'is_summer': 1 if departure_date.month in [6, 7, 8] else 0,
        'is_winter_holidays': 1 if departure_date.month in [12, 1] else 0,
        'is_spring_break': 1 if departure_date.month in [3, 4] else 0,
        'origin_is_hub': 1 if origin in major_hubs else 0,
        'destination_is_hub': 1 if destination in major_hubs else 0,
        'route_popularity': route_popularity.get(f"{origin}_{destination}", 0),
        'is_legacy_carrier': 1 if airline in legacy_carriers else 0,
        'airline_market_share': airline_market_share.get(airline, 0)
    }
    
    # Encode categorical variables
    for col in categorical_columns:
        if col in le_dict:
            if col == 'airline':
                features[f'{col}_encoded'] = le_dict[col].transform([airline])[0] if airline in le_dict[col].classes_ else 0
            elif col == 'origin':
                features[f'{col}_encoded'] = le_dict[col].transform([origin])[0] if origin in le_dict[col].classes_ else 0
            elif col == 'destination':
                features[f'{col}_encoded'] = le_dict[col].transform([destination])[0] if destination in le_dict[col].classes_ else 0
            elif col == 'fare_class':
                features[f'{col}_encoded'] = le_dict[col].transform([fare_class])[0] if fare_class in le_dict[col].classes_ else 0
            elif col == 'booking_category':
                if days_before <= 0:
                    cat = 'same_day'
                elif days_before <= 7:
                    cat = 'week'
                elif days_before <= 14:
                    cat = '2weeks'
                elif days_before <= 30:
                    cat = 'month'
                elif days_before <= 60:
                    cat = '2months'
                elif days_before <= 90:
                    cat = '3months'
                else:
                    cat = 'advance'
                features[f'{col}_encoded'] = le_dict[col].transform([cat])[0] if cat in le_dict[col].classes_ else 0
    
    return [features.get(col, 0) for col in feature_columns]

# Example scenarios
scenarios = [
    {
        'name': 'Early Bird Business Trip',
        'airline': 'American Airlines',
        'origin': 'ORD',
        'destination': 'BOS',
        'booking_date': datetime(2024, 1, 15, 10, 0),
        'departure_date': datetime(2024, 3, 15, 8, 0),
        'fare_class': 'Economy'
    },
    {
        'name': 'Last Minute Weekend Trip',
        'airline': 'American Airlines',
        'origin': 'ORD',
        'destination': 'BOS',
        'booking_date': datetime(2024, 6, 20, 15, 0),
        'departure_date': datetime(2024, 6, 22, 18, 0),
        'fare_class': 'Economy'
    },
    {
        'name': 'Summer Vacation - Business Class',
        'airline': 'American Airlines',
        'origin': 'ORD',
        'destination': 'BOS',
        'booking_date': datetime(2024, 4, 1, 12, 0),
        'departure_date': datetime(2024, 7, 15, 14, 0),
        'fare_class': 'Business'
    }
]

print("\n🎯 Prediction Examples:")
print("="*80)

for scenario in scenarios:
    features = create_prediction_features(**scenario)
    features_array = np.array(features).reshape(1, -1)
    
    predicted_price = best_model['model'].predict(features_array)[0]
    
    print(f"\n✈️ {scenario['name']}:")
    print(f"   Route: {scenario['origin']} → {scenario['destination']}")
    print(f"   Airline: {scenario['airline']}")
    print(f"   Fare Class: {scenario['fare_class']}")
    print(f"   Booking Date: {scenario['booking_date'].strftime('%Y-%m-%d %H:%M')}")
    print(f"   Departure Date: {scenario['departure_date'].strftime('%Y-%m-%d %H:%M')}")
    print(f"   Days in Advance: {(scenario['departure_date'] - scenario['booking_date']).days}")
    print(f"   💰 Predicted Price: ${predicted_price:.2f}")

print(f"\n🔮 All predictions made using: {best_model_name}")

## 9. Business Insights & Recommendations

Key findings and actionable recommendations for travelers and business applications.

In [ ]:
# Business Insights Analysis
print("💼 BUSINESS INSIGHTS & RECOMMENDATIONS")
print("="*60)

# 1. Advance Booking Analysis
advance_booking_analysis = df_processed.groupby('booking_category')['price'].agg(['mean', 'median', 'count']).round(2)
print("\n📅 Advance Booking Strategy:")
print(advance_booking_analysis)

optimal_booking = advance_booking_analysis['mean'].idxmin()
savings_potential = advance_booking_analysis['mean'].max() - advance_booking_analysis['mean'].min()
print(f"\n🎯 Optimal booking window: {optimal_booking}")
print(f"💰 Potential savings: ${savings_potential:.2f}")

# 2. Day of Week Analysis
dow_analysis = df_processed.groupby('departure_day_of_week')['price'].agg(['mean', 'count']).round(2)
dow_labels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_analysis.index = dow_labels
print(f"\n📆 Day of Week Analysis:")
print(dow_analysis)

cheapest_day = dow_analysis['mean'].idxmin()
most_expensive_day = dow_analysis['mean'].idxmax()
dow_savings = dow_analysis['mean'].max() - dow_analysis['mean'].min()
print(f"\n🎯 Cheapest departure day: {cheapest_day}")
print(f"🎯 Most expensive day: {most_expensive_day}")
print(f"💰 Potential savings: ${dow_savings:.2f}")

# 3. Seasonal Analysis
seasonal_analysis = df_processed.groupby('departure_month')['price'].agg(['mean', 'count']).round(2)
print(f"\n🌍 Seasonal Price Analysis:")
print(seasonal_analysis)

cheapest_month = seasonal_analysis['mean'].idxmin()
most_expensive_month = seasonal_analysis['mean'].idxmax()
seasonal_savings = seasonal_analysis['mean'].max() - seasonal_analysis['mean'].min()
print(f"\n🎯 Cheapest month: {cheapest_month}")
print(f"🎯 Most expensive month: {most_expensive_month}")
print(f"💰 Seasonal savings potential: ${seasonal_savings:.2f}")

# 4. Airline Analysis
airline_analysis = df_processed.groupby('airline')['price'].agg(['mean', 'median', 'count']).round(2)
print(f"\n✈️ Airline Price Comparison:")
print(airline_analysis.sort_values('mean'))

# 5. Route Analysis
top_routes = df_processed['route'].value_counts().head(10)
route_price_analysis = df_processed[df_processed['route'].isin(top_routes.index)].groupby('route')['price'].agg(['mean', 'count']).round(2)
print(f"\n🛣️ Top Routes Price Analysis:")
print(route_price_analysis.sort_values('mean'))

# Key Business Recommendations
print(f"\n" + "="*60)
print("🎯 KEY BUSINESS RECOMMENDATIONS")
print("="*60)

recommendations = [
    f"📅 Book flights in the '{optimal_booking}' window for maximum savings (avg ${savings_potential:.0f} less)",
    f"📆 Fly on {cheapest_day}s instead of {most_expensive_day}s to save up to ${dow_savings:.0f}",
    f"🌍 Travel in month {cheapest_month} instead of month {most_expensive_month} for ${seasonal_savings:.0f} savings",
    f"🎫 Business class tickets show different patterns - analyze separately for corporate travel",
    f"🏢 Hub airports (ORD, ATL, etc.) often have more competitive pricing",
    f"⏰ Weekend departures typically cost more - plan weekday travel when possible",
    f"🔄 Use the API's scenario planning feature to compare multiple booking dates",
    f"📊 Monitor price trends 30-60 days before departure for optimal booking timing"
]

for i, rec in enumerate(recommendations, 1):
    print(f"{i:2d}. {rec}")

# Model Performance Summary
print(f"\n" + "="*60)
print("🤖 MODEL PERFORMANCE SUMMARY")
print("="*60)
print(f"🏆 Best Model: {best_model_name}")
print(f"📊 Mean Absolute Error: ${best_model['mae']:.2f}")
print(f"📊 R² Score: {best_model['r2']:.4f}")
print(f"📊 MAPE: {best_model['mape']:.2f}%")
print(f"✅ The model can predict flight prices within ${best_model['mae']:.0f} on average")

# Implementation Notes
print(f"\n" + "="*60)
print("🚀 IMPLEMENTATION NOTES")
print("="*60)
implementation_notes = [
    "The trained model has been saved and can be deployed via the FastAPI",
    "Real-time predictions available through POST /predict endpoint",
    "Scenario planning available through POST /scenario-planning endpoint",
    "Model should be retrained periodically with new data",
    "Consider ensemble methods for improved accuracy",
    "Monitor model drift and performance in production",
    "Implement A/B testing for different model versions"
]

for i, note in enumerate(implementation_notes, 1):
    print(f"{i}. {note}")

print(f"\n🎉 Analysis Complete! The ML system is ready for deployment.")